为了解决CPU执行速度和 IO数据传输速度的严重不匹配

提出了多进程和多线程编程。但是如果一直增加线程会导致 CPU 浪费大量的资源在线程的切换上。进程更是存在可以创建的上限。

最终的结果就是性能严重下降。


因此我们可以使用另一种方法解决，就是异步 IO。我们只需要发出IO指令，然后就去执行其它代码。当IO指令返回结果后再去处理。（消息循环）

## 23.1 协程(Coroutine)

子程序调用总是一个入口，一次返回，调用顺序是明确的。而协程的调用和子程序不同。

协程看上去也是子程序，但执行过程中，在子程序内部可中断，然后转而执行别的子程序，在适当的时候再返回来接着执行。

协程相比线程因为不用消费资源进行线程的切换。因此执行效率更高。

同时不需要担心锁的问题。

协程示例代码如下：

In [ ]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print('[CONSUMER] Consuming %s...' % n)
        r = '200 OK'

def produce(c):
    c.send(None)
    n = 0
    while n < 5:
        n = n + 1
        print('[PRODUCER] Producing %s...' % n)
        r = c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


In [2]:
from time import sleep

def consumer():
    tx = []
    while True:
        rc = yield tx
        print('[CONSUMER] I have reviced %s...' % rc)
        if rc == None:
            continue
        rc.remove('food')
        print('Eating...')
        sleep(1)
        tx = rc
        print('[CONSUMER] I have eaten the food and sent back %s...' % tx)

def producer(c):
    c.send(None)
    tx = ['plate']
    n = 0
    while n < 5:
        n = n + 1
        tx.append('food')
        print('Producing...')
        sleep(1)
        print('[PRODUCER] I am sending %s to comsumer...' % tx)
        rc = c.send(tx)
        if rc == None:
            continue
        print('[PRODUCER] I have recived %s' % rc)
        tx = rc
    c.close()

c = consumer()
producer(c)

Producing...
[PRODUCER] I am sending ['plate', 'food'] to comsumer...
[CONSUMER] I have reviced ['plate', 'food']...
Eating...
[CONSUMER] I have eaten the food and sent back ['plate']...
[PRODUCER] I have recived ['plate']
Producing...
[PRODUCER] I am sending ['plate', 'food'] to comsumer...
[CONSUMER] I have reviced ['plate', 'food']...
Eating...
[CONSUMER] I have eaten the food and sent back ['plate']...
[PRODUCER] I have recived ['plate']
Producing...
[PRODUCER] I am sending ['plate', 'food'] to comsumer...
[CONSUMER] I have reviced ['plate', 'food']...
Eating...
[CONSUMER] I have eaten the food and sent back ['plate']...
[PRODUCER] I have recived ['plate']
Producing...
[PRODUCER] I am sending ['plate', 'food'] to comsumer...
[CONSUMER] I have reviced ['plate', 'food']...
Eating...
[CONSUMER] I have eaten the food and sent back ['plate']...
[PRODUCER] I have recived ['plate']
Producing...
[PRODUCER] I am sending ['plate', 'food'] to comsumer...
[CONSUMER] I have reviced ['plate', 'f

最后套用Donald Knuth的一句话总结协程的特点：

“子程序就是协程的一种特例。”

## 23.2 asyncio

```asyncio```的编程模型就是一个消息循环。```asyncio```模块内部实现了```EventLoop```，把需要执行的协程扔到```EventLoop```中执行，就实现了异步IO。



In [ ]:
%%writefile asyncio_test.py
import asyncio
import threading

# 传入name参数:
async def hello(name):
    # 打印name和当前线程:
    print("Hello %s! (%s)" % (name, threading.current_thread))
    # 异步调用asyncio.sleep(1):
    await asyncio.sleep(1)
    print("Hello %s again! (%s)" % (name, threading.current_thread))
    return name


async def main():
    L = await asyncio.gather(hello("Bob"), hello("Alice"))
    print(L)

asyncio.run(main())


Writing asyncio_test.py


### 小结
asyncio提供了完善的异步IO支持，用asyncio.run()调度一个coroutine；

在一个async函数内部，通过await可以调用另一个async函数，这个调用看起来是串行执行的，但实际上是由asyncio内部的消息循环控制；

在一个async函数内部，通过await asyncio.gather()可以并发执行若干个async函数。